In [ ]:
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-rns2mu8k
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-rns2mu8k
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit b8c0d82d3e8a66063a9fe33ec31c8bb654c1fdc3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.7.0-py3-none-any.whl size=1219268 sha256=baa01a03542e6faf1ebfa91d832d904e072ae5adba5de911073a9c3858d8d842
  Stored in directory: /tmp/pip-ephem-wheel-cache-fhjv5d6u/wheels/93/bb/85/bfec4ee59b2563f74ec87cc2c91c6a4d3e40d3dcdec8ee5afe
Successfully built torch-geometric


In [ ]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import HeteroData
import torch_geometric.transforms as T

In [ ]:
movies = pd.read_csv('https://raw.githubusercontent.com/RizkiYanuar-Tech/TA-1/refs/heads/main/Mungkin%20Fix/Dataset/ml-1m/movies.csv')
movies

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


In [ ]:
ratings = pd.read_csv('https://raw.githubusercontent.com/RizkiYanuar-Tech/TA-1/refs/heads/main/Mungkin%20Fix/Dataset/ml-1m/ratings.csv')
ratings

,UserID,MovieID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [ ]:
users = pd.read_csv('https://raw.githubusercontent.com/RizkiYanuar-Tech/TA-1/refs/heads/main/Mungkin%20Fix/Dataset/ml-1m/users.csv')
users

,UserID,Gender,Age,Occupation
0,1,F,1,10
1,2,M,56,16
2,3,M,25,15
3,4,M,45,7
4,5,M,25,20
...,...,...,...,...
6035,6036,F,25,15
6036,6037,F,45,1
6037,6038,F,56,1
6038,6039,F,45,0


# **Preprocessing**

In [ ]:
all_df = pd.merge(left=users, right=ratings, on='UserID', how='inner')
all_df = pd.merge(left=all_df, right=movies, on='MovieID', how='inner')
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   UserID      1000209 non-null  int64 
 1   Gender      1000209 non-null  object
 2   Age         1000209 non-null  int64 
 3   Occupation  1000209 non-null  int64 
 4   MovieID     1000209 non-null  int64 
 5   Rating      1000209 non-null  int64 
 6   Title       1000209 non-null  object
 7   Genres      1000209 non-null  object
dtypes: int64(5), object(3)
memory usage: 61.0+ MB


In [ ]:
all_df.head()

,UserID,Gender,Age,Occupation,MovieID,Rating,Title,Genres
0,1,F,1,10,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,F,1,10,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,F,1,10,914,3,My Fair Lady (1964),Musical|Romance
3,1,F,1,10,3408,4,Erin Brockovich (2000),Drama
4,1,F,1,10,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [ ]:
print(all_df['Rating'].value_counts())

Rating
4    348971
3    261197
5    226310
2    107557
1     56174
Name: count, dtype: int64


In [ ]:
#Inisialisasi variable
le = LabelEncoder()
le.fit(all_df['Gender']) #Simpan urutan encoding

all_df['Gender'] = le.fit_transform(all_df['Gender'])

In [ ]:
all_df

,UserID,Gender,Age,Occupation,MovieID,Rating,Title,Genres
0,1,0,1,10,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,0,1,10,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,0,1,10,914,3,My Fair Lady (1964),Musical|Romance
3,1,0,1,10,3408,4,Erin Brockovich (2000),Drama
4,1,0,1,10,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...,...,...
1000204,6040,1,25,6,1091,1,Weekend at Bernie's (1989),Comedy
1000205,6040,1,25,6,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,1,25,6,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1,25,6,1096,4,Sophie's Choice (1982),Drama


In [ ]:
all_df.duplicated().sum()

np.int64(0)

In [ ]:
all_df['Genres'] = all_df['Genres'].apply(lambda x: x.split('|')if isinstance(x, str) else [])

# **Knowledge Graph**

In [ ]:
# Mapping ID unik
user_mapping = {id_: i for i, id_ in enumerate(all_df['UserID'].unique())}
movie_mapping = {id_: i for i, id_ in enumerate(all_df['MovieID'].unique())}
gender_mapping = {g: i for i, g in enumerate(all_df['Gender'].unique())}
age_mapping = {a: i for i, a in enumerate(all_df['Age'].unique())}
occupation_mapping = {o: i for i, o in enumerate(all_df['Occupation'].unique())}

# Mengumpulkan genre unik dan buat mapping genre menjadi indeks angka
genres = set()
for genre_list in all_df['Genres']:
    genres.update(genre_list)
genre_mapping = {g: i for i, g in enumerate(sorted(genres))}

In [ ]:
#Representasi Node
num_users = len(user_mapping)
num_movies = len(movie_mapping)
num_genders = len(gender_mapping)
num_ages = len(age_mapping)
num_occupations = len(occupation_mapping)
num_genres = len(genre_mapping)

print(f"Users: {num_users}, Movies: {num_movies}, Genders: {num_genders}, Ages: {num_ages}, Occupations: {num_occupations}, Genres: {num_genres}")

Users: 6040, Movies: 3706, Genders: 2, Ages: 7, Occupations: 21, Genres: 18


In [ ]:
# user → movie edge
user_movie_edge = [
    [user_mapping[row['UserID']], movie_mapping[row['MovieID']]]
    for _, row in all_df.iterrows()
]
user_movie_edge = torch.tensor(user_movie_edge).t().contiguous()
edge_weight = torch.tensor(all_df['Rating'].values, dtype=torch.float)

# user → gender edge
user_gender_edge = [
    [user_mapping[row['UserID']], gender_mapping[row['Gender']]]
    for _, row in all_df.iterrows()
]
user_gender_edge = torch.tensor(user_gender_edge).t().contiguous()

# user → age edge
user_age_edge = [
    [user_mapping[row['UserID']], age_mapping[row['Age']]]
    for _, row in all_df.iterrows()
]
user_age_edge = torch.tensor(user_age_edge).t().contiguous()

# user → occupation edge
user_occupation_edge = [
    [user_mapping[row['UserID']], occupation_mapping[row['Occupation']]]
    for _, row in all_df.iterrows()
]
user_occupation_edge = torch.tensor(user_occupation_edge).t().contiguous()

# movie → genre edge
movie_genre_edge = []
for _, row in all_df.iterrows():
    movie_id = movie_mapping[row['MovieID']]
    for genre in row['Genres']:
        genre_id = genre_mapping[genre]
        movie_genre_edge.append([movie_id, genre_id])
movie_genre_edge = torch.tensor(movie_genre_edge).t().contiguous()

In [ ]:
data = HeteroData()

# Node counts
data['user'].num_nodes = num_users
data['movie'].num_nodes = num_movies
data['gender'].num_nodes = num_genders
data['age'].num_nodes = num_ages
data['occupation'].num_nodes = num_occupations
data['genre'].num_nodes = num_genres

# Edge connections
data['user', 'rates', 'movie'].edge_index = user_movie_edge
data['user', 'rates', 'movie'].edge_weight = edge_weight
data['user', 'has_gender', 'gender'].edge_index = user_gender_edge
data['user', 'has_age', 'age'].edge_index = user_age_edge
data['user', 'has_occupation', 'occupation'].edge_index = user_occupation_edge
data['movie', 'has_genre', 'genre'].edge_index = movie_genre_edge

for ntype in data.node_types:
      data[ntype].node_id = torch.arange(data[ntype].num_nodes)

#Reverse edges untuk pass message
data = T.ToUndirected()(data)

print(data.edge_types)

[('user', 'rates', 'movie'), ('user', 'has_gender', 'gender'), ('user', 'has_age', 'age'), ('user', 'has_occupation', 'occupation'), ('movie', 'has_genre', 'genre'), ('movie', 'rev_rates', 'user'), ('gender', 'rev_has_gender', 'user'), ('age', 'rev_has_age', 'user'), ('occupation', 'rev_has_occupation', 'user'), ('genre', 'rev_has_genre', 'movie')]


# **Splitting Dataset**

In [ ]:
transform = T.RandomLinkSplit(
    num_val=0.10, #10% data validation
    num_test=0.10, #10% data testing
    add_negative_train_samples=False,
    neg_sampling_ratio=0.0,
    edge_types=('user', 'rates', 'movie'),
    rev_edge_types=('movie', 'rev_rates', 'user')
)

train_data, val_data, test_data = transform(data)
total_edges = data['user', 'rates', 'movie'].edge_index.size(1)

print(f"Total Edges: {total_edges}")
print("Training data: ", train_data['user', 'rates', 'movie'].edge_label_index.size(1))
print("Validation data: ", val_data['user', 'rates', 'movie'].edge_label_index.size(1))
print("Testing data: ", test_data['user', 'rates', 'movie'].edge_label_index.size(1))

Total Edges: 1000209
Training data:  800169
Validation data:  100020
Testing data:  100020


In [ ]:
print(train_data.metadata())

(['user', 'movie', 'gender', 'age', 'occupation', 'genre'], [('user', 'rates', 'movie'), ('user', 'has_gender', 'gender'), ('user', 'has_age', 'age'), ('user', 'has_occupation', 'occupation'), ('movie', 'has_genre', 'genre'), ('movie', 'rev_rates', 'user'), ('gender', 'rev_has_gender', 'user'), ('age', 'rev_has_age', 'user'), ('occupation', 'rev_has_occupation', 'user'), ('genre', 'rev_has_genre', 'movie')])


In [ ]:
n_user = data['user'].num_nodes
n_movie = data['movie'].num_nodes
n_interactions = data['user','rates','movie'].edge_index.size(1)

sparsity = 1 - (n_interactions / (n_user * n_movie))
print(f"Sparsity: {sparsity:.4f}")

Sparsity: 0.9553


In [ ]:
torch.save(train_data, 'train_kg.pt')
torch.save(val_data, 'val_kg.pt')
torch.save(test_data, 'test_kg.pt')

In [ ]:
!zip graph_data.zip train_kg.pt val_kg.pt test_kg.pt

  adding: train_kg.pt (deflated 91%)
  adding: val_kg.pt (deflated 92%)
  adding: test_kg.pt (deflated 92%)


In [ ]:
torch.save(data, 'hetero_graph.pt')
print("Graph heterogen berhasil disimpan ke 'hetero_graph.pt'")

Graph heterogen berhasil disimpan ke 'hetero_graph.pt'


In [ ]:
all_df.to_csv("DataMerge.csv", index=False)

In [ ]:
#Simpan mapping dan dataframe
data = {
    'user_mapping': user_mapping,
    'movie_mapping': movie_mapping,
    'gender_mapping': gender_mapping,
    'age_mapping': age_mapping,
    'occupation_mapping': occupation_mapping
}

try:
  with open('mapping.pkl','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
  print("Pickle file saved successfully")
except Exception as e:
  print(f"Error: {e}")

Pickle file saved successfully
